## Import dependencies
```diff
+ You can change below code.
@@ In the report, write which additional packages you used.
```

In [1]:
import numpy as np
import random
import json
import copy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import glob

from tqdm.notebook import tqdm

### Import meta info (tokens, number of users )
```diff
- You should not change below code.
```

In [2]:
meta = json.load(open('./meta.json', 'r'))
tokens = meta['tokens']
num_token = len(tokens)
num_user = meta['num_user']

### Load test dataset 
```diff
- You should not change below code.
@@ Real test dataset will not be distributed. 
@@ For now, Make this notebook work with `valid.json`.
```

In [3]:
test_data = json.load(open('./test.json', 'r'))

### Define Dataset and DataLoader
```diff
- You should not change below code.
- Make your model get the input of sample['token_id']
@@ Even if you batchfy the data when training the model, below code can/should work.
```

In [4]:
class tweetDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        sample['token_id'] = torch.Tensor(sample['token_id'])
        return sample

In [5]:
test_dataset = tweetDataset(test_data)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

sample = next(iter(test_dataloader))

## DEFINE YOUR MODEL IN THE BELOW CELL.
```diff
+ You can change below code.
- Name the class as "Model", DO NOT CHANGE THE NAME OF THE BELOW CLASS! 
@@ In the report, write which model/method you tried, and what is the final model that will be submitted.
```

In [6]:
class Model(nn.Module):
    def __init__(self, num_token, num_user, embed_dim, rnn_dim, num_layers):
        super(Model, self).__init__()
        self.num_token = num_token
        self.num_user = num_user
        self.embed_dim = embed_dim
        self.rnn_dim = rnn_dim
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(num_token, embed_dim)
        self.rnn = nn.LSTM(embed_dim, rnn_dim, num_layers=num_layers, batch_first=True)
        self.out_linear = nn.Linear(rnn_dim, num_user)
        
    def forward(self, token_id):
        embed = self.embedding(token_id)
        out, _ = self.rnn(embed)
        return self.out_linear(out[:, -1])

### Load the best model
```diff
- Name your best model as [FirstName]_[LastName].pth
```

In [7]:
device = 'cuda'

# Name your best model as [FirstName]_[LastName].pth
best_path = './best_baseline.pth'

model = Model(num_token, num_user, embed_dim=512, rnn_dim=1024, num_layers=1).to(device)
model.load_state_dict(torch.load(best_path))
model.eval()

print("Finish loaded model.")

Finish loaded model.


### Number of parameter information
```diff
- The number of parameters should not exceed 20,000,000 !!
- DO NOT USE TRANSFORMER-BASED MODELS!!
@@ Transformer-based models will not be accepted as a submission.
```

In [8]:
num_param = sum(p.numel() for p in model.parameters())
print('Number of parameters: {}'.format(num_param))
print('[NOTE] Number of parameters SHOULD NOT exceed 20,000,000 (20 million).')

Number of parameters: 13152776
[NOTE] Number of parameters SHOULD NOT exceed 20,000,000 (20 million).


### Test the model
```diff
- BELOW CODE CELL SHOULD WORK WITH YOUR MODEL!
@@ Check if your model returns the prediction with shape: BATCH by NUM_USER (NUM_OUTPUT).
```

In [9]:
# Test the model if it generates proper output which shape is B by num_user
pred = model(sample['token_id'].long().to(device))
print('Prediction shape would be BATCH X NUM_OUTPUT : ', pred.shape)

Prediction shape would be BATCH X NUM_OUTPUT :  torch.Size([1, 8])


### Run test
```diff
- Do not change below code!
- Your model should work with below code without any modification.
```

In [10]:
correct_cnt = 0.0
data_cnt = 0.0
for sample in test_dataloader:
    model.eval()

    with torch.no_grad():
        pred = model(sample['token_id'].long().to(device))

    pred_user_id = torch.argmax(pred, dim=-1)

    accu = pred_user_id.detach().cpu() == sample['user_id']

    correct_cnt += torch.sum(accu)
    data_cnt += sample['token_id'].shape[0]

# calculate best valid accuracy
test_accu = (correct_cnt / data_cnt).item()
print('YOUR TEST ACCURACY: {}'.format(test_accu))

YOUR TEST ACCURACY: 0.6848137378692627
